In [1]:
import math
import random

# -----------------------------
# Policy: πθ(RIGHT) = sigmoid(θ)
# -----------------------------
theta = 0.0          # policy parameter
alpha = 0.1          # learning rate
gamma = 0.9          # discount factor

def sigmoid(x):
    return 1 / (1 + math.exp(-x))

def sample_action(theta):
    """Sample action from πθ"""
    p_right = sigmoid(theta)
    return "RIGHT" if random.random() < p_right else "LEFT"

def reward_function(action):
    """Toy reward: RIGHT good, LEFT bad"""
    return 1 if action == "RIGHT" else 0


# -----------------------------
# Step A: Sample one trajectory
# -----------------------------
trajectory = []
T = 3   # episode length

for t in range(T):
    a_t = sample_action(theta)
    r_t1 = reward_function(a_t)

    trajectory.append((a_t, r_t1))

print("Trajectory:", trajectory)


# -----------------------------
# Step B: Compute returns G_t
# -----------------------------
rewards = [r for (_, r) in trajectory]

returns = []
G = 0
for r in reversed(rewards):
    G = r + gamma * G
    returns.insert(0, G)

print("Returns G_t:", returns)


# -----------------------------
# Step C: Compute policy gradient
# g = Σ ∇ log πθ(a_t) * G_t
# -----------------------------
grad = 0
p_right = sigmoid(theta)

for t, ((a_t, r_t1), G_t) in enumerate(zip(trajectory, returns)):

    if a_t == "RIGHT":
        # ∇θ log πθ(RIGHT) = 1 - πθ(RIGHT)
        grad_logp = 1 - p_right
    else:
        # ∇θ log πθ(LEFT) = -πθ(RIGHT)
        grad_logp = -p_right

    grad += grad_logp * G_t

print("Monte Carlo gradient estimate:", grad)


# -----------------------------
# Step D: Update policy parameter
# θ ← θ + α * grad
# -----------------------------
theta = theta + alpha * grad

print("Updated theta:", theta)
print("New π(RIGHT):", sigmoid(theta))

Trajectory: [('LEFT', 0), ('RIGHT', 1), ('LEFT', 0)]
Returns G_t: [0.9, 1.0, 0.0]


Monte Carlo gradient estimate: 0.04999999999999999
Updated theta: 0.004999999999999999
New π(RIGHT): 0.5012499973958399


好的。下面我给你一份完整、严谨、标准符号的 REINFORCE（Monte Carlo Policy Gradient） 推导与算法总结。我会把每一个数学对象的含义、梯度估计、蒙特卡洛平均、以及最终更新式全部写清楚，并给出权威来源（Williams 1992 + Sutton & Barto + OpenAI Spinning Up）。

⸻

✅ REINFORCE：完整严谨版本（Monte Carlo Policy Gradient）

⸻

1. 强化学习 setting（MDP）

我们考虑一个马尔可夫决策过程：

(\mathcal S,\mathcal A,P,R,\gamma)
	•	状态：S_t\in\mathcal S
	•	动作：A_t\in\mathcal A
	•	转移：P(S_{t+1}\mid S_t,A_t)
	•	奖励：R_{t+1}
	•	折扣因子：\gamma\in(0,1)

⸻

2. 参数化随机策略（stochastic policy）

REINFORCE 直接学习策略：

\pi_\theta(a\mid s)

其中 \theta 是策略网络参数（policy parameters）。

⸻

3. Trajectory（轨迹）

策略与环境交互产生一条 episode：

\tau=(S_0,A_0,R_1,S_1,A_1,R_2,\dots,S_T)

轨迹的分布依赖于策略：

\tau\sim\pi_\theta

OpenAI Spinning Up 明确用 trajectories 来定义 policy optimization。
￼

⸻

4. Return（折扣累计回报）

从时间步 t 开始的 return 定义为：

G_t=\sum_{k=0}^{T-t-1}\gamma^k R_{t+1+k}

这就是 Sutton & Barto 的标准 return 定义。
￼

⸻

5. Policy Objective（策略优化目标）

我们希望最大化期望回报：

J(\theta)=\mathbb{E}_{\tau\sim\pi_\theta}[G_0]

目标：

\theta^\*=\arg\max_\theta J(\theta)

⸻

✅ 6. Policy Gradient Theorem（关键梯度形式）

核心结果：

\nabla_\theta J(\theta)
=\mathbb{E}_{\tau\sim\pi_\theta}
\Bigg[
\sum_{t=0}^{T-1}
\nabla_\theta \log \pi_\theta(A_t\mid S_t)\;G_t
\Bigg]

这个形式的意义是：
	•	我们不需要对环境动力学求导
	•	只需要对策略概率的 log 求导

OpenAI Spinning Up 给出同样的 Vanilla Policy Gradient 形式。
￼

⸻

✅ 7. Log-Derivative Trick（数学恒等式）

推导中用到：

\nabla_\theta \pi_\theta(a\mid s)
=\pi_\theta(a\mid s)\nabla_\theta\log\pi_\theta(a\mid s)

这是 score-function / likelihood-ratio trick 的核心。

Williams 1992 就是基于这一思想提出 REINFORCE。
￼

⸻

✅ 8. Monte Carlo Estimator（采样估计）

期望无法解析计算，因此用 N 条轨迹采样：

\tau_1,\dots,\tau_N\sim\pi_\theta

梯度的 Monte Carlo 估计为：

\widehat{\nabla_\theta J(\theta)}
=
\frac{1}{N}
\sum_{i=1}^N
\sum_{t=0}^{T_i-1}
\nabla_\theta\log\pi_\theta(A_{t,i}\mid S_{t,i})\;G_{t,i}

这里：
	•	\frac{1}{N} 就是 Monte Carlo 平均
	•	单条轨迹时 N=1，因此很多教材省略

OpenAI Spinning Up 明确写出“collect trajectories then form a sample estimate”。
￼

⸻

✅ 9. REINFORCE Parameter Update（最终更新）

REINFORCE 使用 stochastic gradient ascent：

\theta \leftarrow \theta+\alpha\widehat{\nabla_\theta J(\theta)}

即：

\theta \leftarrow \theta
+\alpha
\sum_{t=0}^{T-1}
\nabla_\theta\log\pi_\theta(A_t\mid S_t)\;G_t

这就是 REINFORCE Algorithm。

原始论文：Williams 1992 明确称其为 REINFORCE algorithms。
￼

⸻

✅ 10. REINFORCE Algorithm（伪代码）

⸻

Algorithm: REINFORCE (Monte Carlo Policy Gradient)

Repeat for each episode:
	1.	Generate one trajectory:

\tau=(S_0,A_0,R_1,\dots,S_T)
	2.	Compute returns for all time steps:

G_t=\sum_{k\ge 0}\gamma^kR_{t+1+k}
	3.	Update policy parameters:

\theta\leftarrow\theta+\alpha
\sum_{t=0}^{T-1}
\nabla_\theta\log\pi_\theta(A_t\mid S_t)\;G_t

⸻

✅ 11. 最重要的直觉总结

REINFORCE 更新可以读成一句话：

如果某一步动作之后获得的回报 G_t 很大，
就增加该动作在该状态下的概率。

⸻

✅ 12. REINFORCE 的主要特点

特点	说明
On-policy	必须用当前策略采样 trajectory
Monte Carlo	依赖完整 episode return
无偏估计	梯度估计是 unbiased
方差很大	实践中必须用 baseline/advantage 改进

OpenAI 也强调 VPG/REINFORCE 的方差问题，因此实践用 advantage。
￼

⸻

✅ 13. 三个最权威引用
	1.	原始 REINFORCE 论文（必引用）

Williams, 1992
Simple Statistical Gradient-Following Algorithms for Connectionist Reinforcement Learning
￼
	2.	RL 教科书标准定义

Sutton & Barto, Reinforcement Learning: An Introduction (2nd Ed.)
￼
	3.	现代实现与工程视角

OpenAI Spinning Up: Vanilla Policy Gradient / REINFORCE
￼

⸻

✅ 下一步（通向 PPO）

你现在拿到了完整严谨的 REINFORCE。

接下来现代算法（Actor-Critic, PPO）只做两件事：
	1.	用 baseline 降方差：

A_t = G_t - V(S_t)
	2.	用 clipping 稳定更新（PPO）

如果你愿意，我下一步可以同样严谨地给出：

✅ Advantage REINFORCE → Actor-Critic → PPO 的完整数学链条。